# Asynchronous Multi-Sensor Fusion with ROS

In the [Crazyflie ROS Deployment](crazyflie_ros_deployment.ipynb) tutorial, we created a multi-sensor fusion system that processed all sensors **synchronously**—running the Kalman filter at a fixed rate and using the most recent data from each sensor.

**The Real-World Problem**: Sensors don't publish at the same rate!

- Motion capture: 10-120 Hz (high rate, high accuracy)
- Barometer: 10-50 Hz (medium rate, drift-prone)
- IMU: 100-1000 Hz (very high rate, noisy)
- GPS: 1-10 Hz (low rate, outdoor only)
- Camera: 30-60 Hz (medium rate, computationally expensive)

**The Challenge**: How do we fuse sensors that arrive at different times?

This tutorial shows **asynchronous sensor fusion**: running Kalman filter updates only when new measurements arrive, with prediction-only steps in between.

## Synchronous vs Asynchronous Fusion

**Synchronous Fusion** (previous tutorial):
```
Time:  0.00s    0.01s    0.02s    0.03s    0.04s
       |
       |        |        |        |        |
Mocap: ●        ●        ●        ●        ●    (100 Hz conceptually)
Baro:  ●        ●        ●        ●        ●    (100 Hz conceptually)
IMU:   ●        ●        ●        ●        ●    (100 Hz conceptually)
       |
       |        |        |        |        |
KF:    UPDATE   UPDATE   UPDATE   UPDATE   UPDATE
```
- All sensors read at every KF step
- Simple, but unrealistic
- Wastes computation on stale data

**Asynchronous Fusion** (this tutorial):
```
Time:  0.00s    0.01s    0.02s    0.03s    0.04s
       |
       |        |        |        |        |
Mocap: ●                 ●                       (50 Hz)
Baro:  ●             ●             ●            (33 Hz)
IMU:   ●        ●        ●        ●        ●    (100 Hz)
       |
       |        |        |        |        |
KF:    U(M+B+I) U(I)     U(M+I)   U(B+I)   U(I)
```
- Each sensor publishes independently
- KF updates with available measurements
- Prediction-only when no measurements
- Realistic and efficient

Key: **U(M+I)** = Update with Mocap + IMU, **U(I)** = Update with IMU only

## Kalman Filter for Asynchronous Measurements

The standard KF has two steps:

**Prediction** (always runs):
$$
\begin{aligned}
\bar{x}_k &= f(x_{k-1}, u_k) \\
\bar{P}_k &= F_k P_{k-1} F_k^T + Q_k
\end{aligned}
$$

**Update** (runs when measurement available):
$$
\begin{aligned}
K_k &= \bar{P}_k H_k^T (H_k \bar{P}_k H_k^T + R_k)^{-1} \\
x_k &= \bar{x}_k + K_k (y_k - h(\bar{x}_k)) \\
P_k &= (I - K_k H_k) \bar{P}_k
\end{aligned}
$$

**For Asynchronous Fusion**:
- Run **prediction** at high rate (e.g., 100 Hz)
- Run **update** only when a sensor publishes
- Use different $H$ and $R$ for each sensor
- Multiple updates possible in one prediction step!

## Setup: Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pykal import DynamicalSystem
from pykal.ros.ros_node import ROSNode
from pykal.algorithm_library.estimators.kf import KF

# ROS message types
from geometry_msgs.msg import Twist, Vector3
from nav_msgs.msg import Odometry
from std_msgs.msg import Float64
import rclpy
import time

print("Imports successful!")

### System Dynamics (same as before)

In [ ]:
# Crazyflie 3D dynamics
def crazyflie_f(xk, uk, dt):
    """[x, y, z, vx, vy, vz]."""
    pos = xk[:3]
    vel = xk[3:]
    return np.vstack([pos + vel * dt, uk])

def compute_F(dt):
    I3 = np.eye(3)
    return np.block([[I3, I3 * dt], [np.zeros((3, 3)), I3]])

# Individual sensor measurement functions
def h_mocap(xk):
    """Mocap: [x, y, z]."""
    return xk[:3]

def H_mocap():
    return np.array([[1, 0, 0, 0, 0, 0],
                     [0, 1, 0, 0, 0, 0],
                     [0, 0, 1, 0, 0, 0]])

def h_baro(xk):
    """Baro: [z]."""
    return xk[2:3]

def H_baro():
    return np.array([[0, 0, 1, 0, 0, 0]])

def h_imu(xk):
    """IMU: [vx, vy, vz]."""
    return xk[3:]

def H_imu():
    return np.array([[0, 0, 0, 1, 0, 0],
                     [0, 0, 0, 0, 1, 0],
                     [0, 0, 0, 0, 0, 1]])

# Noise covariances
Q = np.diag([0.001, 0.001, 0.001, 0.1, 0.1, 0.1])
R_mocap = np.diag([0.005, 0.005, 0.005])  # 5mm
R_baro = np.array([[0.1]])                 # 10cm
R_imu = np.diag([0.1, 0.1, 0.1])          # 0.1 m/s

print("Dynamics and sensor models defined!")

## Asynchronous Sensor Nodes

First, we create three simulator nodes that publish at **different rates**.

In [ ]:
def create_mocap_sensor_node(rate_hz=50.0):
    """
    Motion capture sensor: 50 Hz, high accuracy.
    """
    # Simulated true state (shared)
    true_state = {'x': np.array([[0.0], [0.0], [1.0], [0.0], [0.0], [0.0]])}
    
    def mocap_callback(tk):
        # Add Gaussian noise
        pos = true_state['x'][:3].flatten()
        noisy = pos + np.random.multivariate_normal(np.zeros(3), R_mocap)
        return {'mocap': noisy}
    
    node = ROSNode(
        node_name='mocap_sensor',
        callback=mocap_callback,
        subscribes_to=[],
        publishes_to=[('mocap', Vector3, '/mocap')],
        rate_hz=rate_hz
    )
    return node, true_state


def create_baro_sensor_node(true_state_ref, rate_hz=25.0):
    """
    Barometer sensor: 25 Hz, medium accuracy.
    """
    def baro_callback(tk):
        z = true_state_ref['x'][2, 0]
        noisy = z + np.random.normal(0, np.sqrt(R_baro[0, 0]))
        return {'baro': np.array([noisy])}
    
    node = ROSNode(
        node_name='baro_sensor',
        callback=baro_callback,
        subscribes_to=[],
        publishes_to=[('baro', Float64, '/baro')],
        rate_hz=rate_hz
    )
    return node


def create_imu_sensor_node(true_state_ref, rate_hz=100.0):
    """
    IMU sensor: 100 Hz, noisy velocity.
    """
    def imu_callback(tk):
        vel = true_state_ref['x'][3:].flatten()
        noisy = vel + np.random.multivariate_normal(np.zeros(3), R_imu)
        return {'imu': noisy}
    
    node = ROSNode(
        node_name='imu_sensor',
        callback=imu_callback,
        subscribes_to=[],
        publishes_to=[('imu', Vector3, '/imu')],
        rate_hz=rate_hz
    )
    return node


print("Asynchronous sensor nodes defined!")
print("  Mocap: 50 Hz")
print("  Baro:  25 Hz") 
print("  IMU:   100 Hz")

## Asynchronous Kalman Filter Node

The key difference: the KF runs at **100 Hz** (IMU rate), but updates only with **available measurements**.

**Implementation Strategy**:
1. Subscribe to all three sensor topics
2. Cache the most recent measurement from each
3. At each callback (100 Hz):
   - **Always** run prediction step
   - Check which sensors have **new data** (timestamp check)
   - Update with only the new measurements
   - Mark measurements as "used"

In [ ]:
def create_async_kalman_filter_node(dt=0.01, rate_hz=100.0):
    """
    Asynchronous Kalman filter.
    
    Runs at 100 Hz but updates only with new measurements.
    """
    # State estimate
    xhat = np.array([[0.0], [0.0], [1.0], [0.0], [0.0], [0.0]])
    P = np.diag([0.1, 0.1, 0.1, 1.0, 1.0, 1.0])
    
    # Track last update times for each sensor
    last_mocap_time = -np.inf
    last_baro_time = -np.inf
    last_imu_time = -np.inf
    
    # Cached measurements
    last_mocap = None
    last_baro = None
    last_imu = None
    
    def filter_callback(tk, mocap, baro, imu, cmd_vel):
        nonlocal xhat, P
        nonlocal last_mocap_time, last_baro_time, last_imu_time
        nonlocal last_mocap, last_baro, last_imu
        
        # Update cached measurements (ROSNode provides latest)
        current_mocap = mocap
        current_baro = baro
        current_imu = imu
        
        # Check which sensors are NEW (simple: check if value changed)
        mocap_new = last_mocap is None or not np.array_equal(current_mocap, last_mocap)
        baro_new = last_baro is None or not np.array_equal(current_baro, last_baro)
        imu_new = last_imu is None or not np.array_equal(current_imu, last_imu)
        
        # === STEP 1: PREDICTION (always) ===
        uk = cmd_vel[:3].reshape(-1, 1)
        Fk = compute_F(dt)
        
        xhat_pred = crazyflie_f(xhat, uk, dt)
        P_pred = Fk @ P @ Fk.T + Q
        
        # === STEP 2: UPDATE (only with new measurements) ===
        xhat_updated = xhat_pred
        P_updated = P_pred
        
        updates_performed = []
        
        # Update with Mocap if new
        if mocap_new:
            y = current_mocap.reshape(-1, 1)
            H = H_mocap()
            R = R_mocap
            
            innovation = y - h_mocap(xhat_updated)
            S = H @ P_updated @ H.T + R
            K = P_updated @ H.T @ np.linalg.pinv(S)
            
            xhat_updated = xhat_updated + K @ innovation
            P_updated = (np.eye(6) - K @ H) @ P_updated
            updates_performed.append('M')
        
        # Update with Baro if new
        if baro_new:
            y = current_baro.reshape(-1, 1)
            H = H_baro()
            R = R_baro
            
            innovation = y - h_baro(xhat_updated)
            S = H @ P_updated @ H.T + R
            K = P_updated @ H.T @ np.linalg.pinv(S)
            
            xhat_updated = xhat_updated + K @ innovation
            P_updated = (np.eye(6) - K @ H) @ P_updated
            updates_performed.append('B')
        
        # Update with IMU if new
        if imu_new:
            y = current_imu.reshape(-1, 1)
            H = H_imu()
            R = R_imu
            
            innovation = y - h_imu(xhat_updated)
            S = H @ P_updated @ H.T + R
            K = P_updated @ H.T @ np.linalg.pinv(S)
            
            xhat_updated = xhat_updated + K @ innovation
            P_updated = (np.eye(6) - K @ H) @ P_updated
            updates_performed.append('I')
        
        # Update state
        xhat = xhat_updated
        P = P_updated
        
        # Update cache
        last_mocap = current_mocap.copy()
        last_baro = current_baro.copy()
        last_imu = current_imu.copy()
        
        # Log updates
        if len(updates_performed) > 0:
            print(f"[{tk:.3f}s] KF updated with: {'+'.join(updates_performed)}")
        else:
            print(f"[{tk:.3f}s] KF prediction only (no new measurements)")
        
        # Convert to Odometry
        pos_est = xhat[:3].flatten()
        vel_est = xhat[3:].flatten()
        estimate = np.concatenate([
            pos_est, [0, 0, 0, 1],
            vel_est, [0, 0, 0]
        ])
        
        return {'estimate': estimate}
    
    node = ROSNode(
        node_name='async_kalman_filter',
        callback=filter_callback,
        subscribes_to=[
            ('/mocap', Vector3, 'mocap'),
            ('/baro', Float64, 'baro'),
            ('/imu', Vector3, 'imu'),
            ('/cmd_vel', Twist, 'cmd_vel'),
        ],
        publishes_to=[('estimate', Odometry, '/estimate')],
        rate_hz=rate_hz,
        stale_config={
            'mocap': {'after': 0.1, 'policy': 'hold'},  # 50 Hz → hold if >100ms
            'baro': {'after': 0.1, 'policy': 'hold'},   # 25 Hz → hold if >100ms
            'imu': {'after': 0.02, 'policy': 'hold'}    # 100 Hz → hold if >20ms
        }
    )
    return node


print("Asynchronous Kalman filter node created!")
print("  Runs at: 100 Hz")
print("  Updates: Only with new measurements")
print("  Strategy: Prediction always, update conditionally")

## Summary

This notebook demonstrated **asynchronous sensor fusion**—a critical skill for real robotics systems where sensors publish at different rates.

**Key Concepts**:

1. **Prediction Always**: Run at high rate (IMU rate)
2. **Update Conditionally**: Only when new measurements arrive
3. **Multiple Updates**: Can update with different sensors at each step
4. **Timestamp Tracking**: Essential for knowing which data is new

**Benefits**:
- Handles multi-rate sensors naturally
- More efficient (no wasted computation)
- Better accuracy (uses measurements when fresh)
- Realistic for hardware deployment

**Next Steps**:
- Implement message buffering for out-of-order arrivals
- Add innovation sequence monitoring
- Handle sensor dropouts gracefully
- Deploy to real hardware with actual async sensors!

---

:doc:`← Python to ROS <index>`